In [1]:
%load_ext lab_black

import altair as alt
import numpy as np
import pandas as pd

# 2. Introducing Scikit-Learn
## Application: Exploring Hand-written Digits

In [2]:
from sklearn.datasets import load_digits

digits = load_digits()

In [3]:
pd.Series(digits.data[:30].tolist()).apply(pd.Series).stack().to_frame(
    "color"
).reset_index().rename(columns={"level_0": "image", "level_1": "index"}).eval(
    "column=index%8"
).eval(
    "row=index//8"
).pipe(
    lambda df: alt.Chart(df, height=80, width=80)
    .mark_rect()
    .encode(
        alt.X("column:O", axis=None),
        alt.Y("row:O", axis=None),
        alt.Color(
            "color:Q",
            legend=None,
            scale=alt.Scale(scheme=alt.SchemeParams("greys", extent=[0, 1])),
        ),
        alt.Facet("image:N", columns=10, title=None),
    )
)

alt.Chart(...)

In [4]:
from sklearn.manifold import Isomap

pd.DataFrame(Isomap().fit_transform(digits.data), columns=["x", "y"]).assign(
    color=digits.target
).pipe(
    lambda df: alt.Chart(df, width=600, height=400)
    .mark_point()
    .encode(x="x:Q", y="y:Q", color="color:N", tooltip=["color:N"])
)

alt.Chart(...)

In [5]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

X_train, X_test, y_train, y_test = train_test_split(
    digits.data, digits.target, random_state=0, test_size=0.25
)
y_pred = GaussianNB().fit(X_train, y_train).predict(X_test)
cm = confusion_matrix(y_test, y_pred)
accuracy_score(y_test, y_pred)

0.8333333333333334

In [6]:
pd.DataFrame(
    cm, columns=digits.target_names, index=digits.target_names
).reset_index().melt(id_vars="index").rename(
    columns={"index": "true", "variable": "predicted", "value": "count"}
).pipe(
    lambda df: alt.Chart(df, height=50, width=50)
    .mark_rect()
    .encode(
        alt.Column("predicted:O", title="Predicted"),
        alt.Row("true:O", title="True"),
        alt.Color("count:Q", scale=alt.Scale(scheme="purples", type="symlog")),
        alt.Tooltip(["count:Q", "true:O", "predicted:O"]),
    )
    .configure_facet(spacing=0)
)

alt.Chart(...)

# 3. Hyperparameters and Model Validation
## Thinking about Model Validation

In [7]:
from scipy import stats
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score, r2_score
from sklearn.model_selection import cross_val_score, LeaveOneOut
from sklearn.neighbors import KNeighborsClassifier

iris = load_iris()
iris

accuracy_score(
    iris.target,
    KNeighborsClassifier(n_neighbors=1).fit(iris.data, iris.target).predict(iris.data),
)

1.0

In [8]:
cross_val_score(KNeighborsClassifier(n_neighbors=1), iris.data, iris.target, cv=5)

array([0.96666667, 0.96666667, 0.93333333, 0.93333333, 1.        ])

In [9]:
stats.describe(
    cross_val_score(
        KNeighborsClassifier(n_neighbors=1), iris.data, iris.target, cv=LeaveOneOut()
    )
)

DescribeResult(nobs=150, minmax=(0.0, 1.0), mean=0.96, variance=0.038657718120805366, skewness=-4.694855340334425, kurtosis=20.041666666666682)

## Selecting the Best Model

In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import learning_curve, validation_curve
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

rng = np.random.RandomState(0)
data = (
    pd.DataFrame({"x": rng.rand(100) ** 2})
    .eval("y = 10 - 1/(x+.1)")
    .assign(y=lambda df: df.y + rng.randn(100))
)

vc = validation_curve(
    make_pipeline(PolynomialFeatures(), LinearRegression()),
    data.x[:, None],
    data.y,
    "polynomialfeatures__degree",
    np.arange(0, 31),
)

(
    alt.Chart(data, title="Data").mark_point().encode(x="x:Q", y="y:Q")
    | (
        pd.DataFrame({"train": np.median(vc[0], 1), "test": np.median(vc[1], 1)})
        .reset_index()
        .melt(id_vars="index")
        .pipe(
            lambda df: alt.Chart(df, title="Validation Curve")
            .mark_line(point=True)
            .encode(
                alt.X("index:O", title="Degree"),
                alt.Y("value:Q", title="Score"),
                alt.Color("variable:N", sort=None),
                alt.Tooltip(["index:O", "variable:N", "value:Q"]),
            )
        )
    )
)

alt.HConcatChart(...)

## Learning Curves

In [11]:
rng = np.random.RandomState(0)
data = (
    pd.DataFrame({"x": rng.rand(1000) ** 2})
    .eval("y = 10 - 1/(x+.1)")
    .assign(y=lambda df: df.y + rng.randn(1000))
)

pd.concat(
    pd.DataFrame(
        {"degree": degree, "train": np.median(lc[1], 1), "test": np.median(lc[2], 1)}
    )
    for degree in np.arange(2, 11)
    if (
        lc := learning_curve(
            make_pipeline(PolynomialFeatures(degree), LinearRegression()),
            data.x[:, None],
            data.y,
            cv=7,
            train_sizes=np.linspace(0.1, 1, 10),
        )
    )
).pipe(
    lambda df: alt.Chart(
        df.reset_index().melt(id_vars=["index", "degree"]), height=150, width=300
    )
    .mark_line(point=True)
    .encode(
        alt.X("index:O", axis=None, title=None),
        alt.Y("value:Q", scale=alt.Scale(zero=False), title="Score"),
        alt.Color("variable:N", sort=None, title="Set"),
        alt.Facet("degree:O", columns=3, title="Degree"),
        alt.Tooltip(["variable:N", "value:Q"]),
    )
)

alt.Chart(...)

# 4. Feature Engineering
## Derived Features

In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

data = pd.DataFrame({"x": [1, 2, 3, 4, 5], "y": [4, 2, 1, 3, 7]}).assign(
    y_pred_linear=(
        lambda data: LinearRegression(fit_intercept=True)
        .fit(data.x[:, None], data.y)
        .predict(data.x[:, None])
    ),
    y_pred_polynomial=(
        lambda data: GridSearchCV(
            make_pipeline(PolynomialFeatures(), LinearRegression(fit_intercept=True)),
            {"polynomialfeatures__degree": range(10)},
            cv=2,
        )
        .fit(data.x[:, None], data.y)
        .best_estimator_.predict(data.x[:, None])
    ),
)

chart = alt.Chart(data).mark_point().encode(x="x:Q", y="y:Q")
(
    (
        chart.properties(title="Linear Regression")
        + chart.mark_line().encode(alt.Y("y_pred_linear:Q", title=None))
    )
    | (
        chart.properties(title="Polynomial Features + Linear Regression")
        + chart.mark_line().encode(alt.Y("y_pred_polynomial:Q", title=None))
    )
).resolve_scale(y="shared")

alt.HConcatChart(...)

# 5. In Depth: Naive Bayes Classification

> Naive Bayes models are a group of extremely fast and simple classification algorithms that are often suitable for very high-dimensional datasets.

> Because they are so fast and have so few tunable parameters, they end up being very useful as a quick-and-dirty baseline for a classification problem.

> Because naive Bayesian classifiers make such stringent assumptions about data, they will generally not perform as well as a more complicated model. That said, they have several advantages:
> - They are extremely fast for both training and prediction
> - They provide straightforward probabilistic prediction
> - They are often very easily interpretable
> - They have very few (if any) tunable parameters

> This means that clusters in high dimensions tend to be more separated, on average, than clusters in low dimensions, assuming the new dimensions actually add information. For this reason, simplistic classifiers like naive Bayes tend to work as well or better than more complicated classifiers as the dimensionality grows: once you have enough data, even a simple model can be very powerful.

\begin{align}
    P(L \mid \mathrm{features}) = \frac{ P(\mathrm{features} \mid L) P(L) }{ P(\mathrm{features}) } \\
\end{align}

\begin{align}
    \frac{ P(L_1 \mid \mathrm{features}) }{ P(L_2 \mid \mathrm{features}) } = \frac{ P(\mathrm{features} \mid L_1)P(L_1) }{ P(\mathrm{features} \mid L_2)P(L_2) }
\end{align}

> Such a model is called a *generative* model because it specifies the hypothetical random process that generates the data.

## Gaussian Naive Bayes

> In this classifier, the assumption is that *data from each label is drawn from a simple Gaussian distribution*.

In [13]:
from sklearn.datasets import make_blobs
from sklearn.naive_bayes import GaussianNB

X, y = make_blobs(100, 2, centers=2, random_state=2, cluster_std=1.5)
chart = (
    lambda data: alt.Chart(data, height=300, width=400)
    .mark_point(filled=True)
    .encode(x="x0", y="x1", color=alt.Color("y:N", legend=None))
)

chart(pd.DataFrame(X).add_prefix("x").assign(y=y)) + (
    chart(
        pd.DataFrame(
            np.array(
                np.meshgrid(
                    np.linspace(X[:, 0].min(), X[:, 0].max()),
                    np.linspace(X[:, 1].min(), X[:, 1].max()),
                )
            ).T.reshape(-1, 2)
        )
        .add_prefix("x")
        .assign(y=lambda df: GaussianNB().fit(X, y).predict(df))
    ).mark_point(filled=True, opacity=0.25)
)

alt.LayerChart(...)

> We see a slightly curved boundary in the classifications—in general, the boundary in Gaussian naive Bayes is quadratic.

## Multinomial Naive Bayes

> Another useful example is multinomial naive Bayes, where the features are assumed to be generated from a simple multinomial distribution. The multinomial distribution describes the probability of observing counts among a number of categories, and thus multinomial naive Bayes is most appropriate for features that represent counts or count rates.

> One place where multinomial naive Bayes is often used is in text classification, where the features are related to word counts or frequencies within the documents to be classified.

In [14]:
from sklearn.datasets import fetch_20newsgroups

data = fetch_20newsgroups()
dir(data), data.target_names

(['DESCR', 'data', 'filenames', 'target', 'target_names'],
 ['alt.atheism',
  'comp.graphics',
  'comp.os.ms-windows.misc',
  'comp.sys.ibm.pc.hardware',
  'comp.sys.mac.hardware',
  'comp.windows.x',
  'misc.forsale',
  'rec.autos',
  'rec.motorcycles',
  'rec.sport.baseball',
  'rec.sport.hockey',
  'sci.crypt',
  'sci.electronics',
  'sci.med',
  'sci.space',
  'soc.religion.christian',
  'talk.politics.guns',
  'talk.politics.mideast',
  'talk.politics.misc',
  'talk.religion.misc'])

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

train, test = (
    pd.Series(data.target_names)
    .pipe(lambda s: s[s.str.contains(r"graphics|religion|space")])
    .pipe(
        lambda categories: (
            fetch_20newsgroups(categories=categories, subset="train"),
            fetch_20newsgroups(categories=categories, subset="test"),
        )
    )
)

model = make_pipeline(TfidfVectorizer(), MultinomialNB()).fit(
    train.data, pd.Series(train.target_names)[train.target]
)

alt.Chart(
    pd.DataFrame(
        confusion_matrix(
            pd.Series(test.target_names)[test.target], model.predict(test.data)
        ),
        columns=train.target_names,
        index=test.target_names,
    )
    .reset_index()
    .melt(id_vars="index")
    .rename(columns={"index": "true", "variable": "predicted", "value": "count"})
).mark_rect(height=100, width=100).encode(
    alt.Column("predicted:O", title="Predicted"),
    alt.Row("true:O", title="True"),
    alt.Color("count:Q", scale=alt.Scale(scheme="purples", type="symlog")),
    alt.Tooltip(["count:Q", "true:O", "predicted:O"]),
).configure_facet(
    spacing=0
)

alt.Chart(...)

In [16]:
pd.Series(
    [
        "sending a payload to the ISS",
        "discussing islam vs atheism",
        "determining the screen resolution",
        "black hole",
        "quasar",
    ]
).to_frame("text").assign(category=lambda df: model.predict(df.text)).pipe(
    lambda df: df.join(
        pd.DataFrame(
            model.predict_proba(df.text), columns=test.target_names
        ).add_prefix("proba: ")
    )
)

,text,category,proba: comp.graphics,proba: sci.space,proba: soc.religion.christian,proba: talk.religion.misc
0,sending a payload to the ISS,sci.space,0.199502,0.421163,0.250665,0.128670
1,discussing islam vs atheism,soc.religion.christian,0.254114,0.145384,0.395239,0.205263
2,determining the screen resolution,comp.graphics,0.581839,0.162995,0.148421,0.106745
3,black hole,soc.religion.christian,0.174071,0.301832,0.423160,0.100938
4,quasar,soc.religion.christian,0.271249,0.275430,0.278216,0.175105


# 6. In Depth: Linear Regression

> Just as naive Bayes is a good starting point for classification tasks, linear regression models are a good starting point for regression tasks. Such models are popular because they can be fit very quickly, and are very interpretable.

## Simple Linear Regression

> A straight-line fit is a model of the form $$ y=ax+b $$ where $a$ is commonly known as the *slope*, and $b$ is commonly known as the *intercept*.

In [17]:
from sklearn.linear_model import LinearRegression

rng = np.random.RandomState(1)
data = pd.DataFrame().eval("x = 10*@rng.rand(50)").eval("y = 2*x - 5 + @rng.randn(50)")
model = LinearRegression().fit(data.x[:, None], data.y)
chart = alt.Chart(data, height=300, width=400).mark_point().encode(x="x", y="y")
x = np.linspace(data.x.min(), data.x.max(), 100)

chart + chart.mark_line().properties(
    data=pd.DataFrame({"x": x, "y": model.predict(x[:, None])})
)

alt.LayerChart(...)

## Basis Function Regression

> One trick you can use to adapt linear regression to nonlinear relationships between variables is to transform the data according to *basis functions*. The idea is to take our multidimensional linear model: $$ y=a_0+a_1x_1+a_2x_2+a_3x_3+\ldots $$ and build the $x_1,x_2,x_3$, and so on, from our single-dimensional input $x$. That is, we let $x_n=f_n(x)$, where $f_n$ is some function that transforms our data.

### Polynomial basis functions

In [18]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

rng = np.random.RandomState(1)
data = (
    pd.DataFrame().eval("x = 10*@rng.rand(50)").eval("y = sin(x) + .1*@rng.randn(50)")
)
model = (
    GridSearchCV(
        make_pipeline(PolynomialFeatures(), LinearRegression()),
        {"polynomialfeatures__degree": range(10)},
    )
    .fit(data.x[:, None], data.y)
    .best_estimator_
)
chart = (
    alt.Chart(data, height=300, width=400).mark_point(filled=True).encode(x="x", y="y")
)
x = np.linspace(data.x.min(), data.x.max(), 1000)

chart + chart.mark_line(interpolate="basis").properties(
    data=pd.DataFrame({"x": x, "y": model.predict(x[:, None])}),
    title=f"Polynomial Features [degree={model['polynomialfeatures'].degree}]",
)

alt.LayerChart(...)

### Gaussian basis functions

In [19]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, validation_curve
from sklearn.pipeline import make_pipeline


class GaussianFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, N=2, width_factor=2.0):
        self.N = N
        self.width_factor = width_factor

    @staticmethod
    def _gauss_basis(x, y, width, axis=None):
        return np.exp(-0.5 * np.sum(((x - y) / width) ** 2, axis))

    def fit(self, X, y=None):
        self.centers_ = np.linspace(X.min(), X.max(), self.N)
        self.width_ = self.width_factor * (self.centers_[1] - self.centers_[0])
        return self

    def transform(self, X):
        return self._gauss_basis(X[:, :, None], self.centers_, self.width_, axis=1)


rng = np.random.RandomState(1)
data = (
    pd.DataFrame().eval("x = 10*@rng.rand(50)").eval("y = sin(x) + .1*@rng.randn(50)")
)
model = (
    GridSearchCV(
        make_pipeline(GaussianFeatures(), LinearRegression()),
        {"gaussianfeatures__N": range(5, 50)},
    )
    .fit(data.x[:, None], data.y)
    .best_estimator_
)
chart = (
    alt.Chart(data, height=300, width=400).mark_point(filled=True).encode(x="x", y="y")
)
x = np.linspace(data.x.min(), data.x.max(), 1000)
gaussianfeatures__N = range(3, 27)

(
    chart
    + chart.mark_line(interpolate="basis").properties(
        data=pd.DataFrame({"x": x, "y": model.predict(x[:, None])}),
        title=f"Gaussian Features [N={model['gaussianfeatures'].N}]",
    )
    | pd.concat(
        map(
            pd.DataFrame,
            validation_curve(
                make_pipeline(GaussianFeatures(), LinearRegression()),
                data.x[:, None],
                data.y,
                "gaussianfeatures__N",
                gaussianfeatures__N,
            ),
        ),
        keys=["train", "test"],
    )
    .pipe(lambda df: df.index.set_levels(gaussianfeatures__N, 1, inplace=True) or df)
    .median(1)
    .reset_index()
    .rename(columns={"level_0": "set", "level_1": "N", 0: "score"})
    .pipe(
        lambda df: alt.Chart(df, title="Validation Curve")
        .mark_line(point=True)
        .encode(
            x="N:O",
            y=alt.Y("score:Q", scale=alt.Scale(zero=False)),
            color=alt.Color("set", sort=None),
            tooltip=["N:O", "score:Q", "set"],
        )
    )
)

alt.HConcatChart(...)

## Regularization

In [20]:
rng = np.random.RandomState(1)
data = (
    pd.DataFrame().eval("x = 10*@rng.rand(50)").eval("y = sin(x) + .1*@rng.randn(50)")
)
model = make_pipeline(GaussianFeatures(30), LinearRegression()).fit(
    data.x[:, None], data.y
)
chart = (
    alt.Chart(data, height=300, width=400).mark_point(filled=True).encode(x="x", y="y")
)
x = np.linspace(data.x.min(), data.x.max(), 1000)

(
    chart
    + chart.mark_line(interpolate="basis").properties(
        data=pd.DataFrame({"x": x, "y": model.predict(x[:, None])}),
        title=f"Gaussian Features [N={model['gaussianfeatures'].N}]",
    )
    & pd.DataFrame(
        {
            "basis location": model["gaussianfeatures"].centers_,
            "coefficient": model["linearregression"].coef_,
        }
    ).pipe(
        lambda df: alt.Chart(df, title="No Regularization")
        .mark_line()
        .encode(x="basis location", y="coefficient")
    )
).resolve_scale(x="shared")

alt.VConcatChart(...)

> We know that such behavior is problematic, and it would be nice if we could limit such spikes expliticly in the model by penalizing large values of the model parameters. Such a penalty is known as *regularization*, and comes in several forms.

### Ridge regression (L2 Regularization)

> Perhaps the most common form of regularization is known as *ridge regression* or *L2 regularization*, sometimes also called *Tikhonov regularization*.
> $$P=\alpha \sum_{n=1}^{N}\theta_n^2 $$ where $α$ is a free parameter that controls the strength of the penalty.

> A **free parameter** is a variable in a mathematical model which cannot be predicted precisely or constrained by the model and must be estimated experimentally or theoretically. A mathematical model, theory, or conjecture is more likely to be right and less likely to be the product of wishful thinking if it relies on few free parameters and is consistent with large amounts of data.
> <br>— [Wikipedia](https://en.wikipedia.org/wiki/Free_parameter)

> One advantage of ridge regression in particular is that it can be computed very efficiently—at hardly more computational cost than the original linear regression model.

In [21]:
from sklearn.linear_model import Ridge

rng = np.random.RandomState(1)
data = (
    pd.DataFrame().eval("x = 10*@rng.rand(50)").eval("y = sin(x) + .1*@rng.randn(50)")
)
model = make_pipeline(GaussianFeatures(30), Ridge(alpha=0.1)).fit(
    data.x[:, None], data.y
)
chart = (
    alt.Chart(data, height=300, width=400).mark_point(filled=True).encode(x="x", y="y")
)
x = np.linspace(data.x.min(), data.x.max(), 1000)

(
    chart
    + chart.mark_line(interpolate="basis").properties(
        data=pd.DataFrame({"x": x, "y": model.predict(x[:, None])}),
        title=f"Gaussian Features [N={model['gaussianfeatures'].N}]",
    )
    & pd.DataFrame(
        {
            "basis location": model["gaussianfeatures"].centers_,
            "coefficient": model["ridge"].coef_,
        }
    ).pipe(
        lambda df: alt.Chart(df, title="L2 Regularization")
        .mark_line()
        .encode(x="basis location", y="coefficient")
    )
).resolve_scale(x="shared")

alt.VConcatChart(...)

### Lasso regression (L1 regularization)

> $$P=\alpha \sum_{n=1}^N |\theta_n|$$
> Though this is conceptually very similar to ridge regression, the results can differ surprisingly: for example, due to geometric reasons lasso regression tends to favor *sparse models* where possible: that is, it preferentially sets model coefficients to exactly zero.

In [22]:
from sklearn.linear_model import Lasso

rng = np.random.RandomState(1)
data = (
    pd.DataFrame().eval("x = 10*@rng.rand(50)").eval("y = sin(x) + .1*@rng.randn(50)")
)
model = make_pipeline(GaussianFeatures(30), Lasso(alpha=1e-2)).fit(
    data.x[:, None], data.y
)
chart = (
    alt.Chart(data, height=300, width=400).mark_point(filled=True).encode(x="x", y="y")
)
x = np.linspace(data.x.min(), data.x.max(), 1000)

(
    chart
    + chart.mark_line(interpolate="basis").properties(
        data=pd.DataFrame({"x": x, "y": model.predict(x[:, None])}),
        title=f"Gaussian Features [N={model['gaussianfeatures'].N}]",
    )
    & pd.DataFrame(
        {
            "basis location": model["gaussianfeatures"].centers_,
            "coefficient": model["lasso"].coef_,
        }
    ).pipe(
        lambda df: alt.Chart(df, title="L1 Regularization")
        .mark_line()
        .encode(x="basis location", y="coefficient")
    )
).resolve_scale(x="shared")

alt.VConcatChart(...)

## Example: Predicting Bicycle Traffic

> In particular, this is an example of how the tools of Scikit-Learn can be used in a statistical modeling framework, in which the parameters of the model are assumed to have interpretable meaning. As discussed previously, this is not a standard approach within machine learning, but such interpretation is possible for some models.

In [23]:
%%bash
mkdir -p data
curl -s 'https://data.seattle.gov/api/views/65db-xm6k/rows.csv?accessType=DOWNLOAD' >data/FremontBridge.csv && head -n3 data/BicycleWeather.csv
echo
curl -s 'https://raw.githubusercontent.com/jakevdp/PythonDataScienceHandbook/master/notebooks/data/BicycleWeather.csv' >data/BicycleWeather.csv && head -n3 data/FremontBridge.csv

STATION,STATION_NAME,DATE,PRCP,SNWD,SNOW,TMAX,TMIN,AWND,WDF2,WDF5,WSF2,WSF5,FMTM,WT14,WT01,WT17,WT05,WT02,WT22,WT04,WT13,WT16,WT08,WT18,WT03
GHCND:USW00024233,SEATTLE TACOMA INTERNATIONAL AIRPORT WA US,20120101,0,0,0,128,50,47,100,90,89,112,-9999,1,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999
GHCND:USW00024233,SEATTLE TACOMA INTERNATIONAL AIRPORT WA US,20120102,109,0,0,106,28,45,180,200,130,179,-9999,-9999,1,-9999,-9999,-9999,-9999,-9999,1,1,-9999,-9999,-9999

Date,Fremont Bridge Total,Fremont Bridge East Sidewalk,Fremont Bridge West Sidewalk
10/03/2012 12:00:00 AM,13,4,9
10/03/2012 01:00:00 AM,10,4,6


In [24]:
bicycle_weather = (
    pd.read_csv("data/BicycleWeather.csv", index_col="DATE", parse_dates=True)
    .pipe(lambda df: df.info() or df)
    .eval("Temperature = (TMIN/10 + TMAX/10)/2")
    .eval("Precipitation = PRCP/254")
    .eval("Dry = 0.+(Precipitation==0)")
    .filter(["Temperature", "Precipitation", "Dry"])
)
bicycle_weather.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1340 entries, 2012-01-01 to 2015-09-01
Data columns (total 25 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   STATION       1340 non-null   object
 1   STATION_NAME  1340 non-null   object
 2   PRCP          1340 non-null   int64 
 3   SNWD          1340 non-null   int64 
 4   SNOW          1340 non-null   int64 
 5   TMAX          1340 non-null   int64 
 6   TMIN          1340 non-null   int64 
 7   AWND          1340 non-null   int64 
 8   WDF2          1340 non-null   int64 
 9   WDF5          1340 non-null   int64 
 10  WSF2          1340 non-null   int64 
 11  WSF5          1340 non-null   int64 
 12  FMTM          1340 non-null   int64 
 13  WT14          1340 non-null   int64 
 14  WT01          1340 non-null   int64 
 15  WT17          1340 non-null   int64 
 16  WT05          1340 non-null   int64 
 17  WT02          1340 non-null   int64 
 18  WT22          1340 non-null   

,Temperature,Precipitation,Dry
DATE,,,
2012-01-01,8.90,0.000000,1.0
2012-01-02,6.70,0.429134,0.0
2012-01-03,9.45,0.031496,0.0
2012-01-04,8.90,0.799213,0.0
2012-01-05,5.85,0.051181,0.0


In [25]:
from pandas.tseries.holiday import USFederalHolidayCalendar

fremont_bridge = (
    pd.read_csv("data/FremontBridge.csv", index_col="Date", parse_dates=True)
    .pipe(lambda df: df.info() or df)
    .resample("d")
    .sum()
    .sum(1)
    .to_frame("Total")
    .assign(Year=lambda df: (df.index - df.index[0]).days / 365)
    .pipe(
        lambda df: df.join(
            pd.get_dummies(df.index.strftime("%a")).astype(float).set_index(df.index)
        )
    )
    .pipe(
        lambda df: df.join(
            pd.Series(
                1.0,
                USFederalHolidayCalendar().holidays(
                    str(df.index.year.min()), str(df.index.year.max())
                ),
                name="Holiday",
            )
        ).fillna({"Holiday": 0.0})
    )
    .pipe(
        lambda df: df.join(
            df.index.map(
                lambda date: (
                    axis := 23.44,
                    latitude := 47.61,
                    days := (date - pd.to_datetime("2000-12-21")).days,
                    m := (
                        1.0
                        - np.tan(np.radians(latitude))
                        * np.tan(np.radians(axis) * np.cos(days * 2 * np.pi / 365.25))
                    ),
                )
                and 24.0 * np.degrees(np.arccos(1 - np.clip(m, 0, 2))) / 180.0,
            ).to_series(index=df.index, name="Daylight")
        )
    )
)
fremont_bridge.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 67128 entries, 2012-10-03 00:00:00 to 2020-04-30 23:00:00
Data columns (total 3 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Fremont Bridge Total          67118 non-null  float64
 1   Fremont Bridge East Sidewalk  67118 non-null  float64
 2   Fremont Bridge West Sidewalk  67118 non-null  float64
dtypes: float64(3)
memory usage: 2.0 MB


,Total,Year,Fri,Mon,Sat,Sun,Thu,Tue,Wed,Holiday,Daylight
Date,,,,,,,,,,,
2012-10-03,7042.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,11.277359
2012-10-04,6950.0,0.002740,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,11.219142
2012-10-05,6296.0,0.005479,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.161038
2012-10-06,4012.0,0.008219,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,11.103056
2012-10-07,4284.0,0.010959,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,11.045208


In [26]:
from sklearn.linear_model import LinearRegression
import sklearn.utils

data = bicycle_weather.join(fremont_bridge, how="inner").sort_index(1)
model = LinearRegression(fit_intercept=False).fit(
    data.drop(columns=["Total"]), data.Total
)
data["Total (predicted)"] = model.predict(data.drop(columns=["Total"]))
data.info()


display(
    alt.Chart(
        data.reset_index().filter(regex=r"index|Total").melt(id_vars="index"),
        height=300,
        width=400,
    )
    .mark_line(opacity=0.5)
    .encode(
        alt.X("index:T", title="Date"),
        alt.Y("value", title="Total"),
        alt.Color("variable", title=None),
    )
)


pd.DataFrame(
    {
        "Effect": model.coef_,
        "Error": np.std(
            [
                model.fit(
                    *sklearn.utils.resample(
                        data.filter(regex=r"^(?!Total)"), data.Total
                    )
                ).coef_
                for _ in range(1000)
            ],
            0,
        ),
    },
    data.filter(regex=r"^(?!Total)").columns,
).sort_values("Effect", ascending=False)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1064 entries, 2012-10-03 to 2015-09-01
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Daylight           1064 non-null   float64
 1   Dry                1064 non-null   float64
 2   Fri                1064 non-null   float64
 3   Holiday            1064 non-null   float64
 4   Mon                1064 non-null   float64
 5   Precipitation      1064 non-null   float64
 6   Sat                1064 non-null   float64
 7   Sun                1064 non-null   float64
 8   Temperature        1064 non-null   float64
 9   Thu                1064 non-null   float64
 10  Total              1064 non-null   float64
 11  Tue                1064 non-null   float64
 12  Wed                1064 non-null   float64
 13  Year               1064 non-null   float64
 14  Total (predicted)  1064 non-null   float64
dtypes: float64(15)
memory usage: 133.0 KB


alt.Chart(...)

,Effect,Error
Tue,1220.467872,168.545653
Wed,1185.347283,167.232513
Dry,1095.397185,68.490354
Mon,1009.765512,175.431886
Thu,964.716230,171.191502
Fri,355.960691,162.770937
Daylight,257.703022,17.566102
Temperature,130.325582,7.106670
Year,53.885426,33.778052
Precipitation,-1329.669764,124.962144
